**Cохранение таблиц из базы в файлы csv для использования в Pandas. Описание схемы базы данных**

# Импорт библиотек, общие настройки

In [1]:
import os
import pandas as pd
import psycopg2

In [2]:
import ipywidgets
from ipywidgets import interact, interact_manual, widgets, Button, DatePicker, Layout
from IPython.display import display, clear_output, HTML

In [3]:
PATH = "./data" # относительный путь к подпапке.
pd.set_option('display.max_columns', None)


In [ ]:
# Close communication with the database
#cursor.close()
#conn.close()

In [5]:
conn = psycopg2.connect("""
    host=----
    port=6432
    dbname=olist
    user=-----
    password=----
    target_session_attrs=read-write
    sslmode=verify-full
""") 

In [6]:
print("Информация о сервере PostgreSQL")
print(conn.get_dsn_parameters(), "\n")

Информация о сервере PostgreSQL
{'user': 'user1', 'channel_binding': 'prefer', 'dbname': 'olist', 'host': 'rc1c-5lsfxv5qer0uol30.mdb.yandexcloud.net', 'port': '6432', 'tty': '', 'options': '', 'sslmode': 'verify-full', 'sslcompression': '0', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'read-write'} 



# Подключение к базе, изучение схемы базы, выгрузка данных для статического первичного анализа структуры информации

In [6]:
cursor = conn.cursor()

In [7]:
# Выполнение SQL-запроса
cursor.execute("SELECT version();")
# Получить результат
record = cursor.fetchone()
print("Вы подключены к - ", record, "\n")

Вы подключены к -  ('PostgreSQL 13.4 (Ubuntu 13.4-201-yandex.50204.1072273362) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',) 



In [8]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema NOT IN ('information_schema','pg_catalog');")
record = cursor.fetchall()
print("Список таблиц - ", record, "\n")

Список таблиц -  [('closed_deals',), ('customers',), ('geolocation',), ('marketing_qualified_leads',), ('order_items',), ('order_payments',), ('order_reviews',), ('orders',), ('products',), ('sellers',)] 



In [38]:
# количество таблиц в базе данных
# len(record)

10

In [39]:
# название таблицы
# print(record[0])
# print(record[0][0])

('closed_deals',)
closed_deals


In [10]:
# Список таблиц и полей
query_txt="SELECT table_name, ordinal_position, column_name FROM information_schema.columns WHERE table_schema NOT IN ('information_schema','pg_catalog');"

cursor.execute(query_txt)
record = cursor.fetchall()
print("Список таблиц и полей - ", record, "\n")

Список таблиц и полей -  [('products', 4, 'product_description_lenght'), ('customers', 3, 'customer_zip_code_prefix'), ('geolocation', 2, 'geolocation_lat'), ('order_items', 2, 'order_item_id'), ('orders', 8, 'order_estimated_delivery_date'), ('products', 1, 'product_id'), ('order_reviews', 6, 'review_creation_date'), ('closed_deals', 13, 'declared_product_catalog_size'), ('customers', 1, 'customer_id'), ('orders', 6, 'order_delivered_carrier_date'), ('products', 3, 'product_name_lenght'), ('closed_deals', 5, 'won_date'), ('products', 7, 'product_length_cm'), ('products', 9, 'product_width_cm'), ('customers', 4, 'customer_city'), ('sellers', 1, 'seller_id'), ('closed_deals', 10, 'has_gtin'), ('closed_deals', 12, 'business_type'), ('order_payments', 3, 'payment_type'), ('orders', 1, 'order_id'), ('products', 8, 'product_height_cm'), ('closed_deals', 2, 'seller_id'), ('order_reviews', 1, 'review_id'), ('orders', 4, 'order_purchase_timestamp'), ('closed_deals', 11, 'average_stock'), ('ord

In [11]:
# создание датафрейма со списком таблиц и полей
df_shema=pd.DataFrame(list(record), columns=['table_name','column_position','column_name']).sort_values(['table_name','column_position'])
df_shema.head(3)


,table_name,column_position,column_name
47,closed_deals,1,mql_id
21,closed_deals,2,seller_id
42,closed_deals,3,sdr_id


In [ ]:
df_shema.to_csv(os.path.join(PATH, 'chema.csv'), sep=';', header=True, index=False)

In [4]:
df_shema = pd.read_csv(os.path.join(PATH, 'chema.csv'), sep=';')

In [5]:
# список таблиц базы
list_of_tables=df_shema['table_name'].unique()

In [14]:
# Количество строк таблиц
rows_in_table={}

for item in list_of_tables:
    #print(item)
    query_txt="SELECT count(*) FROM "+item+";"
    cursor.execute(query_txt)
    record = cursor.fetchone()
    rows_in_table[item]=record[0]
#print("Количество строк - ", record, "\n")
rows_in_table

{'closed_deals': 842,
 'customers': 99441,
 'geolocation': 1000163,
 'marketing_qualified_leads': 8000,
 'order_items': 112650,
 'order_payments': 103886,
 'order_reviews': 99224,
 'orders': 99441,
 'products': 32951,
 'sellers': 3095}

In [19]:
# Сохранение всех таблиц в csv
list_tables_for_upload=list_of_tables # ['geolocation']#,'sellers','products']
for item in list_tables_for_upload:
    query_txt="SELECT * FROM "+item+";"
    cursor.execute(query_txt)
    record = cursor.fetchall()
    col_names = []
    for item_col in cursor.description:
        col_names.append(item_col[0])
    print(item, " Количество строк - ", len(record))
    df1=pd.DataFrame(record, columns=col_names)
    df1.to_csv(os.path.join(PATH, item+'.csv'), sep=';', header=True, index=False)

closed_deals  Количество строк -  842
customers  Количество строк -  99441
geolocation  Количество строк -  1000163
marketing_qualified_leads  Количество строк -  8000
order_items  Количество строк -  112650
order_payments  Количество строк -  103886
order_reviews  Количество строк -  99224
orders  Количество строк -  99441
products  Количество строк -  32951
sellers  Количество строк -  3095


# Описание таблиц и полей

In [6]:
df_table_descr = pd.read_csv(os.path.join(PATH, 'table_description.csv'), sep=';')
df_column_descr = pd.read_csv(os.path.join(PATH, 'column_description.csv'), sep=';')

In [7]:
a5=list(list_of_tables)+['product_category_name_translation']  # формируем список таблиц для выбора
print(a5)
# Интерактивный выбор из списка и показ отфильтрованных строк (или другие действия, которые надо сделать)
@interact
def wwww(w5 = widgets.Dropdown(options=a5, value='closed_deals',description='Таблица:')):
    #pass
    global df_table_descr
    global df_column_descr
    df_filter_table=df_table_descr[df_table_descr['onesoil_table'].str.contains(w5)]
    df_filter_column=df_column_descr[df_column_descr['onesoil_table'].str.contains(w5)]
    display(HTML(df_filter_table.to_html(escape=False)))
    display(HTML( df_filter_column.to_html(escape=False)))

['closed_deals', 'customers', 'geolocation', 'marketing_qualified_leads', 'order_items', 'order_payments', 'order_reviews', 'orders', 'products', 'sellers', 'product_category_name_translation']


interactive(children=(Dropdown(description='Таблица:', options=('closed_deals', 'customers', 'geolocation', 'm…